# Import libraries

In [2]:
import matplotlib.pyplot as plt
import numpy as np

plt.rcParams["font.family"] = "Helvetica"
%matplotlib inline
%config InlineBackend.figure_format = 'svg'

# Energy levels of a single atom 